# FIU DATA

In [10]:
import numpy as np
import os

path = "./data/"
fiu_list = []

for file in os.listdir(path):
    if file.startswith("fiu"):
        fiu_list.append(np.load(path+file))

In [15]:
fiu_array = np.concatenate(fiu_list, 0)

In [20]:
np.save("./fiu_data.npy", fiu_array)

# Typeface Dict

In [23]:
import numpy as np
import os
import urllib.request

path = "./data/"
type_list = []

for file in os.listdir(path):
    if file.startswith("typeface"):
        type_list.append(np.load(path+file).item())

In [27]:
new_type_dict = {}
for dic in type_list:
    for (key, value) in dic.items():
        if new_type_dict.get(key) == None:
            new_type_dict[key] = value
        else:
            continue

In [39]:
import urllib.request
dw_path = "./data/png/"

for (name, url) in new_type_dict.items():
    name = name.replace(' ','_').replace('/','-')
    urllib.request.urlretrieve(url, dw_path+name+".png")

# FastText

In [13]:
import io
fin = io.open("../data/word_emb/wiki-news-300d-1M.vec", 
              'r', encoding='utf-8', newline='\n', errors='ignore')
n, d = map(int, fin.readline().split())

In [16]:
ft_dict = {'u*n*k':0, 'p*a*d':1}
ft_list = ['u*n*k', 'p*a*d']

word2vec = {}
for line in fin:
    tokens = line.rstrip().split(' ')
    word2vec[tokens[0]] = map(float, tokens[1:])
    ft_dict[tokens[0]] = len(ft_list)
    ft_list.append(tokens[0])

np.save('ft_w2v.npy', word2vec)
np.save('ft_w2i.npy', ft_dict)
np.save('ft_i2w.npy', ft_list)

AttributeError: 'str' object has no attribute 'idx2vec'

# Each Typo for an Indexed Text

In [11]:
import nltk
import numpy as np
data_arr = np.load("../data/fiu_data.npy")

In [32]:
UNK = 0
PAD = 1

new_data = []
for (typo_list, text) in data_arr:
    sent_text = nltk.sent_tokenize(text.lower()[2:])
    text = [[word for word in nltk.word_tokenize(sentence)] for sentence in sent_text]
    text = [a for b in text for a in b]
    
    indexed_text = []
    for word in text:
        if ft_dict.get(word) != None:
            indexed_text.append(ft_dict[word])
        else:
            indexed_text.append(UNK)
    
    # cutting till 300 words
    if len(indexed_text) > 300:
        indexed_text = indexed_text[:300]
    elif len(indexed_text) < 300:
        indexed_text += [PAD]*(300-len(indexed_text))

    for typo in typo_list:
        new_data.append([typo] + indexed_text)

In [33]:
np.save('fiu_indexed.npy', new_data)

# Statistics

In [25]:
text_len = []
for (typo, text) in new_data:
    text_len.append(len(text))
print(np.max(text_len), np.mean(text_len), np.median(text_len), np.min(text_len))
print(len(new_data))

4909 273.29222000885346 150.0 55
18072
